# SRS model

This notebook provides analysis for the SRS model using results produced by Mesa simulations.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from model.model import SrsModel

## Parameters

First we need to set up the following model parameters:

| parameter | description | permissible values |
| --- | --- | --- |
| $a$ | number of agents | integral, $n > 0$ |
| $d$ | number of agents sampled for Expansion and Retirement | integral, $0 \leq r \leq 1$ |
| $b$ | probability for novel hypotheses being true | $0 \leq b \leq 1$ |
| $j_{0, N+}$ | base probability for positive novel results being published | $0 \leq j_{0, N+} \leq 1$ |
| $j_{0, N-}$ | base probability for negative novel results being published | $0 \leq j_{0, N-} \leq 1$ |
| $j_{0, R+}$ | base probability for positive replication results being published | $0 \leq j_{0, R+} \leq 1$ |
| $j_{0, R-}$ | base probability for negative replication results being published | $0 \leq j_{0, R-} \leq 1$ |
| $V_{N+}$ | payoff for publishing a novel hypothesis with a positive result | $V_{N+} \geq 0$ |
| $V_{N-}$ | payoff for publishing a novel hypothesis with a negative result | $V_{N-} \geq 0$ |
| $V_{0, R+}$ | base payoff for publishing a replication with a positive result | $V_{0, R+} \geq 0$ |
| $V_{0, R-}$ | base payoff for publishing a replication with a negative result | $V_{0, R-} \geq 0$ |
| $V_{RS}$ | payoff for having an original hypothesis successfully replicated | $V_{RS} \geq 0 $ |
| $V_{RF}$ | payoff for having an original hypothesis unsuccessfully replicated | $V_{RF} \leq 0 $ |
| $\eta_{s}$ | influence of rigour on Productivity Checks | $\eta_s > 0$ |
| $\eta_{r}$ | influence of tallies on replication payoffs | $\eta_r > 0$ |
| $\eta_{j}$ | influence of reputation on publication probabilities | $\eta_j > 0$ |
| $\eta_{N+}$ | influence of reputation on publishing positive novel results | $\eta_{N+} > 0$ |
| $\eta_{N-}$ | influence of reputation on publishing negative novel results | $\eta_{N-} > 0$ |
| $\eta_{R+}$ | influence of reputation on publishing positive replication results | $\eta_{R+} > 0$ |
| $\eta_{R-}$ | influence of reputation on publishing negative replication results | $\eta_{R-} > 0$ |
| $\sigma_t$ | standard deviation for targeted tally replication | $\sigma_{t} > 0$ |
| $\sigma_{c_{N+}}$ | standard deviation for $c_{N+}$ mutation magnitude | $\sigma_{c_{N+}} > 0$ |
| $\sigma_{c_{N-}}$ | standard deviation for $c_{N-}$ mutation magnitude | $\sigma_{c_{N-}} > 0$ |
| $\sigma_{c_{R+}}$ | standard deviation for $c_{R+}$ mutation magnitude | $\sigma_{c_{R+}} > 0$ |
| $\sigma_{c_{R-}}$ | standard deviation for $c_{R-}$ mutation magnitude | $\sigma_{c_{R-}} > 0$ |

In [2]:
a = 1000
d = 10
b = 0.01
j_0_N_pos = 0.9
j_0_N_neg = 0.6
j_0_R_pos = 0.7
j_0_R_neg = 0.4
v_N_pos = 1
v_N_neg = 0.8
v_0_R_pos = 0.5
v_0_R_neg = 0.5
v_RS = 0.2
v_RF = -0.8
eta_s = 0.2
eta_r = 0.5
eta_j = 0.5
eta_N_pos = 0.2
eta_N_neg = 0.2
eta_R_pos = 0.2
eta_R_neg = 0.2
sigma_t = 0.1
sigma_c_N_pos = 0.01
sigma_c_N_neg = 0.01
sigma_c_R_pos = 0.01
sigma_c_R_neg = 0.01

## Intializing the SRS model simulation

Next, we initialize an SRS model instance with the parameters we've set.

In [3]:
model = SrsModel(
    a,
    d,
    b,
    j_0_N_pos,
    j_0_N_neg,
    j_0_R_pos,
    j_0_R_neg,
    v_N_pos,
    v_N_neg,
    v_0_R_pos,
    v_0_R_neg,
    v_RS,
    v_RF,
    eta_s,
    eta_r,
    eta_j,
    eta_N_pos,
    eta_N_neg,
    eta_R_pos,
    eta_R_neg,
    sigma_t,
    sigma_c_N_pos,
    sigma_c_N_neg,
    sigma_c_R_pos,
    sigma_c_R_neg,
)


## Initializing agents

Now we need to initialize a set of agents in our model. To keep things simple, we start all agents with the same set of parameters for their research strategies. These parameters are as follows:

| parameter | description | permissible values |
| --- | --- | --- |
| $\gamma$ | the power of the agents | $0 \leq \gamma \leq 1$ |
| $\tau$ | the rigour of the agents | $\tau > 0$ |
| $r$ | probability that an agent chooses to replicate a published hypothesis | $0 \leq r \leq 1$ |
| $c_{N+}$ | probability that an agent chooses to publish a  positive novel result | $0 \leq c_{N+} \leq 1$ |
| $c_{N-}$ | probability that an agent chooses to publish a  negative novel result | $0 \leq c_{N-} \leq 1$ |
| $c_{R+}$ | probability that an agent chooses to publish a  positive replication result | $0 \leq c_{R+} \leq 1$ |
| $c_{R-}$ | probability that an agent chooses to publish a  negative replication result | $0 \leq c_{R-} \leq 1$ |

In [4]:
gamma = 0.8
tau = 100
r = 0.2
c_N_pos = 1
c_N_neg = 0.8
c_R_pos = 0.6
c_R_neg = 0.4

In [5]:
model.initialize_agents(gamma, tau, r, c_N_pos, c_N_neg, c_R_pos, c_R_neg)